Code to convert a cocos format annotation file to h5 format that AlphaPose uses

In [5]:
import h5py
from pycocotools.coco import COCO
import numpy as np
import pandas as pd

In [4]:
print (h5py.__version__)

2.9.0


Inspect a final file

In [3]:
f = h5py.File('annot_coco_foot_t.h5', 'r')

In [155]:
list(f.keys())

['bndbox', 'imgname', 'part']

In [156]:
dset1x = f['imgname']
dset2x = f['bndbox']
dset3x = f['part']

Open json annotation files and extract information

In [6]:
dataDir='/home/hearth/ml/data/'
dataType='train2017'
annFile_full='{}/annotations/person_keypoints_{}.json'.format(dataDir,dataType)
annFile_toes='{}/annotations/foot_{}.json'.format(dataDir,dataType)

In [10]:
def extract_info(annFile):
    coco=COCO(annFile)
    cats = coco.loadCats(coco.getCatIds())
    nms=[cat['name'] for cat in cats]
    #print('COCO categories: \n{}\n'.format(' '.join(nms)))

    nms = set([cat['supercategory'] for cat in cats])
    #print('COCO supercategories: \n{}'.format(' '.join(nms)))

    catIds = coco.getCatIds(catNms=['person']);
    imgIds = coco.getImgIds(catIds=catIds );

    annIds = coco.getAnnIds(imgIds=imgIds)
    anns = coco.loadAnns(annIds)

    imgs = [anns[x]['image_id'] for x in range(len(anns))]
    imgs = [str(img).rjust(12, '0')+'.jpg' for img in imgs]

    conv =[np.asarray([ord(ch) for ch in img]) for img in imgs]
    bbox = [np.asarray(anns[x]['bbox']) for x in range(len(anns))]

    conv = np.array(conv)
    keypoints = [anns[x]['keypoints'] for x in range(len(anns))]

    return (imgs, conv, bbox, keypoints)

In [16]:
imgs, conv, bbox, keypts = extract_info(annFile_toes)

loading annotations into memory...
Done (t=0.45s)
creating index...
index created!


In [17]:
keypts= np.array(keypts)

In [18]:
bbox = np.array(bbox)

In [134]:
f = h5py.File("annot_coco_foot_train.h5", "w")

In [19]:
bbox=bbox.reshape((14758,1,4))

In [135]:
dsetx = f.create_dataset("part", data=keypts)

In [136]:
dsety = f.create_dataset("imgname", data=conv)

In [150]:
dsetz = f.create_dataset("bndbox", data=bbox)

In [152]:
f.close()

## If combining/merging two dataset is required (Not required for CMU toes dataset)

In [16]:
percentile_list1 = pd.DataFrame(
    {'imgs': imgs,
     'bbox': bbox,
     'keypts': keypoints
    })

In [67]:
percentile_list1[['b1','b2', 'b3', 'b4']] = pd.DataFrame(percentile_list1.bbox.values.tolist(), index= percentile_list1.index)
#print (percentile_list1)

In [68]:
merged = pd.merge(percentile_list1, percentile_list, on=['b1', 'b2', 'b3', 'b4'])

In [74]:
#merged['combined'] = merged['keypts_x']+merged['keypts_y']

In [93]:
keypts = [np.array([keypoints[x][i:i + 2] for i in range(0, len(keypoints[x]), 3)]) for x in range(len(keypoints))] 

In [ ]:
keypts = [np.array([merged['combined'][x][i:i + 2] for i in range(0, len(merged['combined'][x]), 3)]) for x in range(len(merged['combined']))] 

In [ ]:
merged['keypts_y'][0]

In [ ]:
len(merged['combined'][0])

In [ ]:
merged.loc[merged['imgs_x'] == '000000317433.jpg']

In [ ]:
percentile_list1.loc[percentile_list1['imgs'] == '000000317433.jpg']['keypts']